<a href="https://colab.research.google.com/github/di5han/Concept-and-Technologies-of-AI/blob/main/Worksheet4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# Load the Titanic dataset
data = pd.read_csv("/content/drive/MyDrive/dataset/Titanic-Dataset.csv")
# Drop all categorical columns except 'Survived'
categorical_columns = data.select_dtypes(include=['object']).columns
data = data.drop(columns=[col for col in categorical_columns if col != 'Survived'])
# Check for missing values
missing_info = data.isnull().sum() / len(data) * 100
# Handle missing values
for column in data.columns:
    if missing_info[column] > 10: # If more than 10% missing
        data[column] = data[column].fillna(data[column].mean())
    else: # If less than 10% missing
        # For dropna, direct assignment handles it better than inplace=True
        # This line drops rows where the specific column has NaN if the percentage is low
        # Reassigning the result ensures changes are reflected in the DataFrame
        data = data.dropna(subset=[column])
# Display cleaned data
print("Data after processing:\n", data.head())
print("\nMissing values after processing:\n", data.isnull().sum())

Data after processing:
    PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare
0            1         0       3  22.0      1      0   7.2500
1            2         1       1  38.0      1      0  71.2833
2            3         1       3  26.0      0      0   7.9250
3            4         1       1  35.0      1      0  53.1000
4            5         0       3  35.0      0      0   8.0500

Missing values after processing:
 PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64


In [2]:
import numpy as np
# Separate features (X) and target (y)
X = data.drop(columns=['Survived']).values # Convert features to NumPy array
y = data['Survived'].values # Convert target to NumPy array
# Define a function for train-test split from scratch
def train_test_split_scratch(X, y, test_size=0.3, random_seed=42):
    """
    Splits dataset into train and test sets.

    Arguments:
    X : np.ndarray
        Feature matrix.
    y : np.ndarray
        Target array.
    test_size : float
        Proportion of the dataset to include in the test split (0 < test_size < 1).
    random_seed : int
        Seed for reproducibility.

    Returns:
    X_train, X_test, y_train, y_test : np.ndarray
        Training and testing splits of features and target.
    """
    np.random.seed(random_seed)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices) # Shuffle the indices
    test_split_size = int(len(X) * test_size)
    test_indices = indices[:test_split_size]
    train_indices = indices[test_split_size:]

    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split_scratch(X, y, test_size=0.3)

# Output shapes to verify
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (624, 6)
Shape of X_test: (267, 6)
Shape of y_train: (624,)
Shape of y_test: (267,)


In [3]:
def euclidean_distance(point1, point2):
    """
    Calculate the Euclidean distance between two points in n-dimensional space.
    Arguments:
    point1 : np.ndarray
    The first point as a numpy array.
    point2 : np.ndarray
    The second point as a numpy array.
    Returns:
    float
    The Euclidean distance between the two points.
    Raises:
    ValueError: If the input points do not have the same dimensionality.
    """
    # Check if the points are of the same dimension
    if point1.shape != point2.shape:
        raise ValueError("Points must have the same dimensions to calculate Euclidean distance.")
    # Calculate the Euclidean distance
    distance = np.sqrt(np.sum((point1 - point2) ** 2))
    return distance

In [4]:
# Test case for the function
try:
    # Define two points
    point1 = np.array([3, 4])
    point2 = np.array([0, 0])
    # Calculate the distance
    result = euclidean_distance(point1, point2)
    # Check if the result matches the expected value (e.g., sqrt(3^2 + 4^2) = 5)
    expected_result = 5.0
    assert np.isclose(result, expected_result), f"Expected {expected_result}, but got {result}"
    print("Test passed successfully!")
except ValueError as ve:
    print(f"ValueError: {ve}")
except AssertionError as ae:
    print(f"AssertionError: {ae}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Test passed successfully!


In [5]:
# Function for KNN prediction for a single query
def knn_predict_single(query, X_train, y_train, k=3):
    """
    Predict the class label for a single query using the K-nearest neighbors algorithm.
    Arguments:
    query : np.ndarray
    The query point for which the prediction is to be made.
    X_train : np.ndarray
    The training feature matrix.
    y_train : np.ndarray
    The training labels.
    k : int, optional
    The number of nearest neighbors to consider (default is 3).
    Returns:
    int
    The predicted class label for the query.
    """
    distances = [euclidean_distance(query, x) for x in X_train]
    sorted_indices = np.argsort(distances)
    nearest_indices = sorted_indices[:k]
    nearest_labels = y_train[nearest_indices]
    prediction = np.bincount(nearest_labels).argmax()
    return prediction

In [6]:
# Function to test KNN for all test samples
def knn_predict(X_test, X_train, y_train, k=3):
    """
    Predict the class labels for all test samples using the K-nearest neighbors algorithm.
    Arguments:
    X_test : np.ndarray
    The test feature matrix.
    X_train : np.ndarray
    The training feature matrix.
    y_train : np.ndarray
    The training labels.
    k : int, optional
    The number of nearest neighbors to consider (default is 3).
    Returns:
    np.ndarray
    An array of predicted class labels for the test samples.
    """
    predictions = [knn_predict_single(x, X_train, y_train, k) for x in X_test]
    return np.array(predictions)

In [7]:
# Test case for KNN on the Titanic dataset
# Assume X_train, X_test, y_train, and y_test have been prepared using the code above
try:
# Define the test set for the test case
X_test_sample = X_test[:5] # Taking a small subset for testing
y_test_sample = y_test[:5] # Corresponding labels for the subset
# Make predictions
predictions = knn_predict(X_test_sample, X_train, y_train, k=3)
# Print test results
print("Predictions:", predictions)
print("Actual labels:", y_test_sample)
# Check if predictions match expected format
assert predictions.shape == y_test_sample.shape, "The shape of predictions does not match the
shape of the actual labels."
print("Test case passed successfully!")
except AssertionError as ae:
print(f"AssertionError: {ae}")
except Exception as e:
print(f"An unexpected error occurred: {e}")

SyntaxError: unterminated string literal (detected at line 13) (ipython-input-1768431233.py, line 13)

In [ ]:
# Function to compute accuracy of predictions
def compute_accuracy(y_true, y_pred):
"""
Compute the accuracy of predictions.
Arguments:
y_true : np.ndarray ; The true labels.
y_pred : np.ndarray; The predicted labels.
Returns:
float : The accuracy as a percentage (0 to 100).
"""
correct_predictions = np.sum(y_true == y_pred)
total_predictions = len(y_true)
accuracy = (correct_predictions / total_predictions) * 100
return accuracy

In [ ]:
# Perform prediction on the entire test set
try:
# Make predictions on the entire test set
predictions = knn_predict(X_test, X_train, y_train, k=3)
# Compute the accuracy
accuracy = compute_accuracy(y_test, predictions)
# Print the accuracy
print(f"Accuracy of the KNN model on the test set: {accuracy:.2f}%")
except Exception as e:
print(f"An unexpected error occurred during prediction or accuracy computation: {e}")

In [ ]:
# Function to test KNN on different values of k and plot the accuracies
import matplotlib.pyplot as plt
def experiment_knn_k_values(X_train, y_train, X_test, y_test, k_values):
"""
Run KNN predictions for different values of k and plot the accuracies.
Arguments:
X_train : np.ndarray
The training feature matrix.
y_train : np.ndarray
The training labels.
X_test : np.ndarray
The test feature matrix.
y_test : np.ndarray
The test labels.
k_values : list of int
A list of k values to experiment with.
Returns:
dict
A dictionary with k values as keys and their corresponding accuracies as values.
"""
accuracies = {}
for k in k_values:
# Make predictions using the current value of k
predictions = knn_predict(X_test, X_train, y_train, k=k)
# Compute the accuracy
accuracy = compute_accuracy(y_test, predictions)
accuracies[k] = accuracy
print(f"Accuracy for k={k}: {accuracy:.2f}%")
# Plot the accuracies
plt.figure(figsize=(10, 5))
plt.plot(k_values, list(accuracies.values()), marker=’o’)
plt.xlabel(’k (Number of Neighbors)’)
plt.ylabel(’Accuracy (%)’)
plt.title(’Accuracy of KNN with Different Values of k’)
plt.grid(True)
plt.show()
return accuracies

In [ ]:
# Define the range of k values to experiment with
k_values = range(1, 21) # You can adjust this range as needed
# Run the experiment
try:
accuracies = experiment_knn_k_values(X_train, y_train, X_test, y_test, k_values)
print("Experiment completed. Check the plot for the accuracy trend.")
except Exception as e:
print(f"An unexpected error occurred during the experiment: {e}")